## 環境設定

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install tensorflow==2.11.0

## 写真へのタグ付け

参考
[WD14Taggerによるタグ付け](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_README-ja.md#wd14tagger%E3%81%AB%E3%82%88%E3%82%8B%E3%82%BF%E3%82%B0%E4%BB%98%E3%81%91)

画像の入っているフォルダを指定すると同じフォルダ内にタグ付けされたテキストファイルが作成される

In [2]:
!python finetune/tag_images_by_wd14_tagger.py \
    "./train/20_usu frog" \
    --batch_size=4

2023-11-13 14:10:40.738168: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-13 14:10:40.794586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 14:10:41.496931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
using existing wd14 tagger model
2023-11-13 14:10:44.847370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

### タグ付けされたデータのフォルダ名の変更

このスクリプトでは下記の`2.DreamBooth、キャプション方式（正則化画像使用可）`形式を使用します。

[学習データの準備について
](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_README-ja.md#%E5%AD%A6%E7%BF%92%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E6%BA%96%E5%82%99%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6)

上記からの引用↓

*`画像ごとにキャプションが記録されたテキストファイルを用意して学習します。たとえば特定のキャラを学ばせると、画像の詳細をキャプションに記述することで（白い服を着たキャラA、赤い服を着たキャラA、など）キャラとそれ以外の要素が分離され、より厳密にモデルがキャラだけを学ぶことが期待できます。`*

学習画像を使用する際.tomlファイルを指定せずコマンドラインオプションで指定する方法を採用しています。運用状況に応じて適宜適切な方を選択してください。

[以前の指定形式（設定ファイルを用いずコマンドラインから指定）
](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_README-ja.md#%E4%BB%A5%E5%89%8D%E3%81%AE%E6%8C%87%E5%AE%9A%E5%BD%A2%E5%BC%8F%E8%A8%AD%E5%AE%9A%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%92%E7%94%A8%E3%81%84%E3%81%9A%E3%82%B3%E3%83%9E%E3%83%B3%E3%83%89%E3%83%A9%E3%82%A4%E3%83%B3%E3%81%8B%E3%82%89%E6%8C%87%E5%AE%9A)


#### コマンドラインから指定する際のフォルダ名の設定

[step 1. 学習用画像の準備](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_README-ja.md#step-1-%E5%AD%A6%E7%BF%92%E7%94%A8%E7%94%BB%E5%83%8F%E3%81%AE%E6%BA%96%E5%82%99)

画像とキャプションが入っているフォルダ名を以下のように設定します。

`<繰り返し回数>_<identifier> <class>`

`<identifier>`はLoRAで学習させたい特定のインスタンスを表す記述子です。今回のケースでは一般的なカエルではなく、鳥獣戯画スタイルのカエルを学習します。

その場合identifierに"tyoju_zaga"のように入れても良いのですが、その言葉事態がすでに一般的に使われている名称であるとバッティングしてしまい不都合なため、世の中に使われていない適当な文字列を使います。3文字程度のsls,とかcpcというような文字列を使います。

今回のケースでは
`20_sls frog`
のように設定し。一般のfrogクラスに属する特殊なsls(鳥獣戯画)を学習することとします。

繰り返し回数については以下を参照してください。今回は正則化画像を使わないため重要ではありません

[繰り返し回数について](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_README-ja.md#%E7%B9%B0%E3%82%8A%E8%BF%94%E3%81%97%E5%9B%9E%E6%95%B0%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6)



## LoRAの学習

下記のセルのコマンドでLoRAを実行する。`sdxl_train_network.py`を実行する際に `--network_module=networks.lora`と指定することでLoRAファインチューニングが実行される

オプションの詳細は[学習の実行](https://github.com/kohya-ss/sd-scripts/blob/main/docs/train_network_README-ja.md#%E5%AD%A6%E7%BF%92%E3%81%AE%E5%AE%9F%E8%A1%8C)を参考

In [ ]:
!accelerate launch --num_cpu_threads_per_process 4 sdxl_train_network.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0"\
    --train_data_dir="train" \
    --resolution 512 --output_dir=output --output_name=frog \
    --train_batch_size=1 --learning_rate=1e-4 --max_train_epochs 20 \
    --mixed_precision=fp16 --save_precision=fp16 \
    --seed 42 --save_model_as=safetensors --save_every_n_epochs=1 \
    --optimizer_type="adafactor" \
    --max_data_loader_n_workers=4 \
    --network_module=networks.lora --network_dim=4 \
    --training_comment="add any comment" \
    --caption_extension=".txt" \
    --cache_latents \
    --no_half_vae

学習が始まると`save_every_n_epochs`で指定した頻度で`output_dir`で指定したディレクトリにLoRAファイル(.safetensors)が出力されます。ファイル名のプレフィックスは`output_name`で指定できます。

### 推論　